In [ ]:
import py2neo
py2neo.__version__

## Create

### Step 0: Connect to Graph using Py2Neo

In [ ]:
from py2neo import Graph
graph = Graph("bolt://localhost:11003", password='1234')

### Step 1: Create Node and Relationship Subgraphs using Py2Neo

This is just a sample -- the full dataset is quite long and would be silly to keep here ([full create data](py2neo-movie-graph-data.ipynb)), it's at the end of this notebook. It'll be necessary to have all the data to run some of the examples.

[Full Create](py2neo-movie-graph-data.ipynb)

In [ ]:
from py2neo import Node, Relationship

# Nodes
TheMatrix = Node("Movie", title='The Matrix', released=1999, tagline='Welcome to the Real World')
Keanu = Node("Person", name='Keanu Reeves', born=1964)
Carrie = Node("Person", name='Carrie-Anne Moss', born=1967)
Laurence = Node("Person", name='Laurence Fishburne', born=1961)
Hugo = Node("Person", name='Hugo Weaving', born=1960)
LillyW = Node("Person", name='Lilly Wachowski', born=1967)
LanaW = Node("Person", name='Lana Wachowski', born=1965)
JoelS = Node("Person", name='Joel Silver', born=1952)
Emil = Node("Person", name="Emil Eifrem", born=1978)

# Relationships
LillyWTheMatrix = Relationship(LillyW, "DIRECTED", TheMatrix)
LanaWTheMatrix = Relationship(LanaW, "DIRECTED", TheMatrix)
JoelSTheMatrix = Relationship(JoelS, "PRODUCED", TheMatrix)
KeanuTheMatrix = Relationship(Keanu, "ACTED_IN", TheMatrix)
KeanuTheMatrix['roles'] = ['Neo']
CarrieTheMatrix = Relationship(Carrie, "ACTED_IN", TheMatrix)
CarrieTheMatrix['roles'] = ['Trinity']
LaurenceTheMatrix = Relationship(Laurence, "ACTED_IN", TheMatrix)
LaurenceTheMatrix['roles'] = ['Morpheus']
HugoTheMatrix = Relationship(Hugo, "ACTED_IN", TheMatrix)
HugoTheMatrix['roles'] = ['Agent Smith']
EmilTheMatrix = Relationship(Emil, "ACTED_IN", TheMatrix)
EmilTheMatrix['roles'] = ['Emil']

### Step 2: Commit using Py2Neo

In [ ]:
tx = graph.begin()
tx.create(TheMatrix)
tx.create(Keanu)
tx.create(Carrie)
tx.create(Laurence)
tx.create(Hugo)
tx.create(LillyW)
tx.create(LanaW)
tx.create(JoelS)
tx.create(Emil)
tx.create(KeanuTheMatrix)
tx.create(CarrieTheMatrix)
tx.create(LaurenceTheMatrix)
tx.create(HugoTheMatrix)
tx.create(LillyWTheMatrix)
tx.create(LanaWTheMatrix)
tx.create(JoelSTheMatrix)
tx.create(EmilTheMatrix)
# tx.commit()

## Find

In [ ]:
from py2neo.matching import NodeMatcher
node_matcher = NodeMatcher(graph)
keanu = node_matcher.match("Person", name="Keanu Reeves").first()
keanu

Demo based on: [docs](<https://py2neo.org/v5/database.html#py2neo.database.Graph.nodes)

In [ ]:
keanu0 = graph.nodes[1]
keanu1 = graph.nodes.get(1)
keanu2 = graph.nodes.match("Person", name="Keanu Reeves").first()
    
assert(keanu0 == keanu1 == keanu2)
keanu0 == keanu1 == keanu2

In [ ]:
match_using_matcher = node_matcher.match(name="Keanu Reeves").first()
match_using_graphnodes = graph.nodes.match(name="Keanu Reeves").first()

assert match_using_matcher == match_using_graphnodes
match_using_matcher == match_using_graphnodes

In [ ]:
len(graph.nodes.match("Person"))

#### Find the actor named "Tom Hanks"...

In [ ]:
node_matcher.match(name="Tom Hanks").first()

In [ ]:
graph.nodes.match(name="Tom Hanks").first()

#### Find the movie with title "Cloud Atlas"...

In [ ]:
node_matcher.match(title="Cloud Atlas").first()

In [ ]:
graph.nodes.match(title="Cloud Atlas").first()

#### Find 10 people...

In [ ]:
node_matcher.match("Person").limit(10).all()

In [ ]:
graph.nodes.match("Person").limit(10).all()

#### Find movies released in the 1990s...

In [ ]:
graph.nodes.match("Movie").where('_.released >= 1990', '_.released < 2000').all()

In [ ]:
node_matcher.match("Movie").where('_.released >= 1990', '_.released < 2000').all()

### Query

#### List all Tom Hanks movies...

In [ ]:
tom = graph.nodes.match(name="Tom Hanks").first()
graph.match(nodes=[tom], r_type="ACTED_IN").all()

In [ ]:
graph.relationships.match(nodes=[tom], r_type="ACTED_IN").all()

#### Who directed "Cloud Atlas"?

In [ ]:
results = graph.run('MATCH (cloudAtlas {title: "Cloud Atlas"})<-[:DIRECTED]-(directors) RETURN directors.name').data()
results

In [ ]:
results = graph.run('MATCH (cloudAtlas {title: "Cloud Atlas"})<-[:DIRECTED]-(directors) RETURN directors.name')
results.data()

In [ ]:
cloudAtlas = graph.nodes.match(title="Cloud Atlas").first()
directors = graph.match(r_type="DIRECTED", nodes=(None, cloudAtlas)) # << see notes about use of nodes=() here
for director in directors:
    print(director.nodes[0]['name'])

#### Tom Hanks' co-actors...

In [ ]:
results = graph.run('MATCH (tom:Person {name:"Tom Hanks"})-[:ACTED_IN]->(m)<-[:ACTED_IN]-(coActors) RETURN coActors.name').data()
results

In [ ]:
results = graph.run('MATCH (tom:Person {name:"Tom Hanks"})-[:ACTED_IN]->(m)<-[:ACTED_IN]-(coActors) RETURN coActors.name')
results.data()

#### How people are related to "Cloud Atlas"...

In [ ]:
results = graph.run('MATCH (people:Person)-[relatedTo]-(:Movie {title: "Cloud Atlas"}) RETURN people.name, Type(relatedTo), relatedTo')
results.to_table()

people.name,Type(relatedTo),relatedTo
Halle Berry,ACTED_IN,"(Halle Berry)-[:ACTED_IN {roles: ['Luisa Rey', 'Jocasta Ayrs', 'Ovid', 'Meronym']}]->(_64)"
Stefan Arndt,PRODUCED,(Stefan Arndt)-[:PRODUCED {}]->(_64)
Hugo Weaving,ACTED_IN,"(Hugo Weaving)-[:ACTED_IN {roles: ['Bill Smoke', 'Haskell Moore', 'Tadeusz Kesselring', 'Nurse Noakes', 'Boardman Mephi', 'Old Georgie']}]->(_64)"
Lilly Wachowski,DIRECTED,(Lilly Wachowski)-[:DIRECTED {}]->(_64)
Tom Tykwer,DIRECTED,(Tom Tykwer)-[:DIRECTED {}]->(_64)
Tom Hanks,ACTED_IN,"(Tom Hanks)-[:ACTED_IN {roles: ['Zachry', 'Dr. Henry Goose', 'Isaac Sachs', 'Dermot Hoggins']}]->(_64)"
Jim Broadbent,ACTED_IN,"(Jim Broadbent)-[:ACTED_IN {roles: ['Vyvyan Ayrs', 'Captain Molyneux', 'Timothy Cavendish']}]->(_64)"
Lana Wachowski,DIRECTED,(Lana Wachowski)-[:DIRECTED {}]->(_64)
David Mitchell,WROTE,(David Mitchell)-[:WROTE {}]->(_64)


In [ ]:
results = graph.run('MATCH (people:Person)-[relatedTo]-(:Movie {title: "Cloud Atlas"}) RETURN people.name, Type(relatedTo), relatedTo')
results.to_data_frame()

,people.name,Type(relatedTo),relatedTo
0,Halle Berry,ACTED_IN,"{'roles': ['Luisa Rey', 'Jocasta Ayrs', 'Ovid'..."
1,Stefan Arndt,PRODUCED,{}
2,Hugo Weaving,ACTED_IN,"{'roles': ['Bill Smoke', 'Haskell Moore', 'Tad..."
3,Lilly Wachowski,DIRECTED,{}
4,Tom Tykwer,DIRECTED,{}
5,Tom Hanks,ACTED_IN,"{'roles': ['Zachry', 'Dr. Henry Goose', 'Isaac..."
6,Jim Broadbent,ACTED_IN,"{'roles': ['Vyvyan Ayrs', 'Captain Molyneux', ..."
7,Lana Wachowski,DIRECTED,{}
8,David Mitchell,WROTE,{}


In [ ]:
results = graph.run('MATCH (people:Person)-[relatedTo]-(:Movie {title: "Cloud Atlas"}) RETURN people.name, Type(relatedTo), relatedTo')
results.to_series()

#### Movies and actors up to 4 "hops" away from Kevin Bacon

In [ ]:
results = graph.run('MATCH (bacon:Person {name:"Kevin Bacon"})-[*1..4]-(hollywood) RETURN DISTINCT hollywood')
results.data()

In [ ]:
results = graph.run('MATCH (bacon:Person {name:"Kevin Bacon"})-[*1..4]-(hollywood) RETURN DISTINCT hollywood')
len(results.data())

#### Bacon path, the shortest path of any relationships to Meg Ryan

In [ ]:
results = graph.run('MATCH p=shortestPath((bacon:Person {name:"Kevin Bacon"})-[*]-(meg:Person {name:"Meg Ryan"})) RETURN p')
results.data()

### Recommend

#### Extend Tom Hanks co-actors, to find co-co-actors who haven't worked with Tom Hanks...

In [ ]:
results = graph.run('MATCH (tom:Person {name:"Tom Hanks"})-[:ACTED_IN]->(m)<-[:ACTED_IN]-(coActors), (coActors)-[:ACTED_IN]->(m2)<-[:ACTED_IN]-(cocoActors) WHERE NOT (tom)-[:ACTED_IN]->()<-[:ACTED_IN]-(cocoActors) AND tom <> cocoActors RETURN cocoActors.name AS Recommended, count(*) AS Strength ORDER BY Strength DESC')
results.data()

#### Find someone to introduce Tom Hanks to Tom Cruise

In [ ]:
results = graph.run('MATCH (tom:Person {name:"Tom Hanks"})-[:ACTED_IN]->(m)<-[:ACTED_IN]-(coActors), (coActors)-[:ACTED_IN]->(m2)<-[:ACTED_IN]-(cruise:Person {name:"Tom Cruise"}) RETURN tom, m, coActors, m2, cruise')
results.data()

In [ ]:
graph.delete_all()

In [ ]:
len(graph.match())

In [ ]:
graph.match().all()